In [1]:
from django.db import models


class Book(models.Model):
    author = models.CharField(max_length=100)

In [2]:
class FooOrBar(type):
    def __new__(cls, name, base, attrs, *args, **kwargs):
        if 'foo' in attrs and 'bar' in attrs:
            raise TypeError('class cannot contain both foo and bar')
        if 'foo' not in attrs and 'bar' not in attrs:
            raise TypeError('class must provide either a foo or a bar')
        return super(FooOrBar, cls).__new__(cls, name, base, attrs, *args, **kwargs)
    
    
class Valid1(metaclass=FooOrBar):
    foo = 42

In [3]:
class Valid(metaclass=FooOrBar):
    foo = 42
    bar = 45

TypeError: class cannot contain both foo and bar

In [4]:
import six


@six.add_metaclass(FooOrBar)
class Valid2:
    pass

TypeError: class must provide either a foo or a bar

In [7]:
@six.add_metaclass(FooOrBar)
class ValidP:
    foo = 42
    
    
class ValidC(ValidP):
    pass

In [8]:
class FooOrBar1(type):
    def __new__(cls, *args, **kwargs):
        super_ = super(FooOrBar1, cls).__new__(cls, *args, **kwargs)
        if hasattr(super_, 'foo') and hasattr(super_, 'bar'):
            raise TypeError('class cannot contain both foo and bar')
        if not hasattr(super_, 'foo') and not hasattr(super_, 'bar'):
            raise TypeError('class must provide either a foo or a bar')
        return super_

In [11]:
@six.add_metaclass(FooOrBar1)
class ValidP:
    foo = 42
    
    
class ValidC(ValidP):
    bar = 20

TypeError: class cannot contain both foo and bar

### 非继承属性

In [6]:
class Meta(type):
    def __new__(cls, name, bases, sttrs, *args, **kwargs):
        if sttrs.get('abstract', False):
            return super(Meta, cls).__new__(cls, name, bases, sttrs, *args, **kwargs)


class AbstractClass(metaclass=Meta):
    abstract = True


class RegularClass(AbstractClass):
    abstract = False

In [8]:
class Logged(type):
    def __new__(cls, name, bases, attrs):
        for key, value in attrs.items():
            if callable(value):
                attrs[key] = cls.log_call(value)
        return super(Logged, cls).__new__(cls, name, bases, attrs)
    
    @staticmethod
    def log_call(func):
        def inner(*args, **kwargs):
            print('调用')
            try:
                respone = func(*args, **kwargs)
                print('successs')
                return respone
            except Exception as exc:
                print('exception')
                raise 
        return inner

In [9]:
class Myclass(metaclass=Logged):
    def foo(self):
        pass
    
    def bar(self):
        raise TypeError("test")

Myclass().foo()

调用
successs


In [10]:
Myclass().bar()

调用
exception


TypeError: test